In [18]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings('ignore')

# Apriori: Predicting potential products that could be cross-sell with energy drinks which are the main items in most invoices 

In [19]:
transaction = pd.read_csv("customerTransaction.csv")

In [20]:
transaction.head()

,Customer Code,Customer Name,Customer Type,DC Name,District Name Eng,District Name TH,Last Date,Latitude,Longitude,Province Name Eng,...,GroupNameLevel1,GroupNameLevel2,Invoice Date,Invoice Number,Net Amount,Product Code,Product Name TH,Unit Price,Month,Year
0,FMK000122,Route * 12356 ตู้7,โชว์ห่วยเล็ก (Community),เอฟเอ็มกาฬสินธุ์,Mueang Kalasin,เมืองกาฬสินธุ์,2022-08-05 06:10:36.177,16.435583,103.524766,Kalasin,...,M-150,ENERGY DRINK,20220202,HSA10C00523,1985.98,10000600,เอ็ม-150 ไฮ วิตามินบี12 150มล. 1x5x10,10.5,2,2022
1,FMK000122,Route * 12356 ตู้7,โชว์ห่วยเล็ก (Community),เอฟเอ็มกาฬสินธุ์,Mueang Kalasin,เมืองกาฬสินธุ์,2022-08-05 06:10:36.177,16.435583,103.524766,Kalasin,...,M-150,ENERGY DRINK,20220613,HSA10C02228,79.44,10000586,เอ็ม-150 กระชายดำ BK 100มล. 1x5x10,8.5,6,2022
2,FMK000122,Route * 12356 ตู้7,โชว์ห่วยเล็ก (Community),เอฟเอ็มกาฬสินธุ์,Mueang Kalasin,เมืองกาฬสินธุ์,2022-08-05 06:10:36.177,16.435583,103.524766,Kalasin,...,C-VITT,REFRESHMENT,20220722,HSA10C03115,124.61,10000627,ซี-วิท เลมอน 10ปี 140มล 1x3x10,13.5,7,2022
3,FMK000122,Route * 12356 ตู้7,โชว์ห่วยเล็ก (Community),เอฟเอ็มกาฬสินธุ์,Mueang Kalasin,เมืองกาฬสินธุ์,2022-08-05 06:10:36.177,16.435583,103.524766,Kalasin,...,LIPO,ENERGY DRINK,20220422,HSA10C01447,485.98,10000512,ลิโพวิตัน-ดี 100มล 1X5X10,10.5,4,2022
4,FMK000122,Route * 12356 ตู้7,โชว์ห่วยเล็ก (Community),เอฟเอ็มกาฬสินธุ์,Mueang Kalasin,เมืองกาฬสินธุ์,2022-08-05 06:10:36.177,16.435583,103.524766,Kalasin,...,C-VITT,REFRESHMENT,20220316,HSA10C00986,373.83,10000603,ซี-วิท ออเร้นจ์ 140 มล. 1x3x10,13.5,3,2022


In [21]:
product_master = pd.read_excel('Master_Products.xlsx')
product_master.head()

,productcode,m_fam_product_name,sku_name_thai,sku_name_english,brand,category,Product Name Sales Tools,Is3M
0,10000718,คาลพิส โซดา ซากูระ พีช 245มล.,คาลพิส โซดา ซากูระ พีช 245มล.,CALPIS LACTO Sakura Peach 245 ml,CALPIS LACTO,REFRESHMENT,คาลพิส โซดา ซากูระ พีช 245มล.,Other
1,10000571,คาลพิส แลคโตะ โซดา กลิ่นส้มยูซุ 245มล,คาลพิส แลคโตะ โซดา กลิ่นส้มยูซุ 245มล,CALPIS LACTO Soda Yuzu Honey 245 ml,CALPIS LACTO,REFRESHMENT,คาลพิส โซดา ยูสุ ฮันนี่ 245มล 1X4X6,Other
2,10000573,คาลพิส เมเปิ้ลไซรัป 300มล,คาลพิส เมเปิ้ลไซรัป 300มล,CALPIS LACTO Maple Syrup 300 ml,CALPIS LACTO,REFRESHMENT,คาลพิส เมเปิ้ลไซรัป 300มล 1X24,Other
3,10000420,คาลพิส ออริจินัล 300 มล,คาลพิส ออริจินัล 300 มล,CALPIS LACTO Original 300 ml,CALPIS LACTO,REFRESHMENT,คาลพิส ออริจินัล 300 มล. 1x4x6 SBM,Other
4,10000309,คาลพิส แลคโตะ มินิ ออริจินัล 140มล,คาลพิส แลคโตะ มินิ ออริจินัล 140มล,CALPIS LACTO Mini Original 140 ml,CALPIS LACTO,REFRESHMENT,คาลพิส ออริจินัล น้ำตาลน้อย 140มล1x5x6,Other


In [22]:
# create a dictionary to convert product name TH to ENG later
product_name_TH_ENG = product_master.groupby('productcode')['sku_name_english'].unique().reset_index()
product_name_TH_ENG = product_name_TH_ENG.append({'productcode':0, 'sku_name_english':'NA'}, ignore_index=True)

product_code = product_name_TH_ENG['productcode'].tolist()
product_name_ENG = product_name_TH_ENG['sku_name_english'].tolist()

product_name_TH_ENG_dict = {product_code[i]: product_name_ENG[i] for i in range(len(product_code))}

In [23]:
# remove 25761 rows where 'Base Qty' == 0 
transactionBQ = transaction[transaction['Base Qty']>0]

In [24]:
# check transactions percentage of each province
round(transactionBQ['Province Name Eng'].value_counts()/len(transactionBQ)*100,1)

Bangkok                     23.0
Nakhon Ratchasima           12.6
Ubon Ratchathani             9.8
Udon Thani                   6.4
Kalasin                      6.0
Samut Sakhon                 4.7
Nonthaburi                   4.3
Sa kaeo                      3.9
Songkhla                     3.5
Prachin Buri                 3.1
Phichit                      3.0
Lop Buri                     2.4
Nakhon Sawan                 2.2
Samut Prakarn                2.1
Saraburi                     2.0
Lampang                      1.9
Phra Nakhon Si Ayutthaya     1.6
Kamphaeng Phet               1.5
Phitsanulok                  1.5
Chai Nat                     1.0
Uthai Thani                  0.9
Phatthalung                  0.8
Sing Buri                    0.8
Phetchabun                   0.7
Satun                        0.1
Name: Province Name Eng, dtype: float64

In [25]:
# check sales percentage of each province
round(transactionBQ.groupby('Province Name Eng')['Net Amount'].sum()/transactionBQ['Net Amount'].sum()*100,1).sort_values(ascending=False)

Province Name Eng
Bangkok                     23.7
Nakhon Ratchasima            9.5
Ubon Ratchathani             8.6
Nonthaburi                   6.4
Udon Thani                   4.9
Nakhon Sawan                 4.7
Kalasin                      4.5
Samut Sakhon                 4.4
Songkhla                     4.1
Phichit                      3.5
Saraburi                     2.9
Phra Nakhon Si Ayutthaya     2.9
Sa kaeo                      2.7
Lop Buri                     2.5
Prachin Buri                 2.5
Samut Prakarn                2.0
Uthai Thani                  2.0
Lampang                      1.6
Phitsanulok                  1.5
Kamphaeng Phet               1.3
Chai Nat                     1.1
Phatthalung                  0.9
Phetchabun                   0.8
Sing Buri                    0.8
Satun                        0.1
Name: Net Amount, dtype: float64

In [26]:
# consider only Province with transaction counts >= 3% & total net amount >= 3%
majorIndex = round(transactionBQ['Province Name Eng'].value_counts()/len(transactionBQ)*100,1).index
majorIndex = np.array(majorIndex)

majorString = ''

for i in majorIndex:
    majorString += i
    majorString += '|'
    
test = transactionBQ[transactionBQ['Province Name Eng'].str.contains(majorString[:-1])]
test['Province Name Eng'].nunique()

25

In [27]:
# we dont need quantity sum 
# we need either has taken or not 
# so if user has taken that item mark as 1 else mark as 0

def convert_into_binary(x):
    if x > 0:
        return 1
    else:
        return 0

In [28]:
rules_mlxtend_all_provinces = pd.DataFrame()

for province in majorIndex:
    
    # limit to 40 recommendation per province, not 10 because will remove M-150 and duplicates 
    count10 = 0
    
    test_province = test[test['Province Name Eng']==province]
    
    # convert data in format which is required 
    basket = pd.pivot_table(data=test_province, index='Invoice Number', columns='Product Code', values='Base Qty', aggfunc='sum', fill_value=0)
    
    basket_sets = basket.applymap(convert_into_binary)
    
    # Find the min_support. I'm looking at the 25 Percentile for frequency of Product Code 
    total_transaction = transactionBQ['Invoice Number'].nunique()
    frequencyOfItemAt25Percentile = transactionBQ['Product Code'].value_counts().describe()[4]
    min_supportcalculted = frequencyOfItemAt25Percentile / total_transaction * 100
    
    # call apriori function
    # min_support refers to how many times the item will appear. eg 0.07 means ocurs 7 times out of 100 transactions 
    frequent_itemsets = apriori(basket_sets, min_support=min_supportcalculted, use_colnames=True)
    
    # A lift value greater than 1 means that item Y is likely to be bought if item X is bought, while a value less than 1 means that item Y is unlikely to be bought if item X is bought.
    # Apply association rules on frequent itemset
    # An antecedent is an item found within the data. 
    # A consequent is an item found in combination with the antecedent
    rules_mlxtend = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    rules_mlxtend.sort_values(by=['lift'], ascending=False, inplace=True)
    rules_mlxtend = rules_mlxtend.head(40)
    rules_mlxtend = rules_mlxtend.iloc[::2]
    rules_mlxtend.insert(1, "antecedents_2", 0)
    rules_mlxtend.insert(3, "consequents_2", 0)
    rules_mlxtend.insert(0, "Province Name Eng", 'NA')
#     rules_mlxtend['Product Name ENG'] = 'NA'
#     rules_mlxtend['Product Name ENG_2'] = 'NA'
    rules_mlxtend['Product Name TH'] = 'NA'
    rules_mlxtend['Product Name TH_2'] = 'NA'
    rules_mlxtend['GroupNameLevel1'] = 'NA'
    rules_mlxtend['GroupNameLevel1_2'] = 'NA'
    
    # convert frozenset to integers, remove groupnamelevel1 == 'M-150'
    for i in range(len(rules_mlxtend.index)):
        
        # add province name
        rules_mlxtend.at[rules_mlxtend.index[i], 'Province Name Eng'] = province
    
        # convert antecedents into integer
        current_antecedent = rules_mlxtend.iloc[i].antecedents

        if len(current_antecedent) == 1:
            antecedent = int(str(current_antecedent)[11:-2])
            rules_mlxtend.at[rules_mlxtend.index[i],'antecedents']= antecedent
            # add in corresponding GroupNamelevel1
            rules_mlxtend.at[rules_mlxtend.index[i],'GroupNameLevel1'] = transactionBQ[transactionBQ['Product Code']==antecedent]['GroupNameLevel1'].iloc[0]

        else:
            antecedent =  int(str(current_antecedent)[11:19])
            antecedent_2 = int(str(current_antecedent)[20:29])
            rules_mlxtend.at[rules_mlxtend.index[i],'antecedents']= antecedent
            rules_mlxtend.at[rules_mlxtend.index[i],'antecedents_2']= antecedent_2
             # add in corresponding GroupNamelevel1
            rules_mlxtend.at[rules_mlxtend.index[i],'GroupNameLevel1'] = transactionBQ[transactionBQ['Product Code']==antecedent]['GroupNameLevel1'].iloc[0]
            rules_mlxtend.at[rules_mlxtend.index[i],'GroupNameLevel1_2'] = transactionBQ[transactionBQ['Product Code']==antecedent_2]['GroupNameLevel1'].iloc[0]



        # turn consequents into integer, add GroupNameLevel1, add product name TH
        current_consequent = rules_mlxtend.iloc[i].consequents

        if len(current_consequent) == 1:
            consequent = int(str(current_consequent)[11:-2])
            rules_mlxtend.at[rules_mlxtend.index[i],'consequents']= consequent
            groupnamelevel = transactionBQ[transactionBQ['Product Code']==consequent]['GroupNameLevel1'].iloc[0]
            # add in corresponding Product Name TH
            rules_mlxtend.at[rules_mlxtend.index[i],'Product Name TH'] = transactionBQ[transactionBQ['Product Code']==consequent]['Product Name TH'].iloc[0]
            # add in corresponding GroupNamelevel1
            rules_mlxtend.at[rules_mlxtend.index[i],'GroupNameLevel1'] = transactionBQ[transactionBQ['Product Code']==consequent]['GroupNameLevel1'].iloc[0]

        else:
            consequent =  int(str(current_consequent)[11:19])
            consequent_2 = int(str(current_consequent)[20:29])
            rules_mlxtend.at[rules_mlxtend.index[i],'consequents']= consequent
            rules_mlxtend.at[rules_mlxtend.index[i],'consequents_2']= consequent_2
            # add in corresponding Product Name TH
            rules_mlxtend.at[rules_mlxtend.index[i],'Product Name TH'] = transactionBQ[transactionBQ['Product Code']==consequent]['Product Name TH'].iloc[0]
            rules_mlxtend.at[rules_mlxtend.index[i],'Product Name TH_2'] = transactionBQ[transactionBQ['Product Code']==consequent]['Product Name TH'].iloc[0]
             # add in corresponding GroupNamelevel1
            rules_mlxtend.at[rules_mlxtend.index[i],'GroupNameLevel1'] = transactionBQ[transactionBQ['Product Code']==consequent]['GroupNameLevel1'].iloc[0]
            rules_mlxtend.at[rules_mlxtend.index[i],'GroupNameLevel1_2'] = transactionBQ[transactionBQ['Product Code']==consequent_2]['GroupNameLevel1'].iloc[0]

            
        # limit to 40 recommendation per province
        count10 += 1
        
        if count10 == 40:
            break
    
    # convert product name TH to product name ENG
    rules_mlxtend['sku_name_english'] = rules_mlxtend['consequents'] 
    rules_mlxtend['sku_name_english_2'] = rules_mlxtend['consequents_2']
    
    
    rules_mlxtend = rules_mlxtend[['Province Name Eng', 'antecedents', 'antecedents_2', 'consequents', 'consequents_2', 'lift', 'confidence', 'Product Name TH', 'Product Name TH_2', 'sku_name_english', 'sku_name_english_2', 'GroupNameLevel1', 'GroupNameLevel1_2']]
    rules_mlxtend = rules_mlxtend[(rules_mlxtend.GroupNameLevel1 != 'M-150') & (rules_mlxtend.GroupNameLevel1_2 != 'M-150')].head(10)
    rules_mlxtend_all_provinces = pd.concat([rules_mlxtend_all_provinces, rules_mlxtend], ignore_index=True)

The IF component of an association rule is known as the antecedent. The THEN component is known as the consequent.

In [29]:
# convert productcode to sku_name_english
rules_mlxtend_all_provinces['sku_name_english'] = rules_mlxtend_all_provinces['sku_name_english'].map(product_name_TH_ENG_dict)
rules_mlxtend_all_provinces['sku_name_english_2'] = rules_mlxtend_all_provinces['sku_name_english_2'].map(product_name_TH_ENG_dict)

In [30]:
rules_mlxtend_all_provinces.to_csv('AprioriResults.csv',index=False)

In [31]:
check = pd.read_csv('AprioriResults.csv')
check.head()

,Province Name Eng,antecedents,antecedents_2,consequents,consequents_2,lift,confidence,Product Name TH,Product Name TH_2,sku_name_english,sku_name_english_2,GroupNameLevel1,GroupNameLevel1_2
0,Bangkok,10000600,10000604,10000602,10000603,7.074331,0.300657,ซี-วิท เลมอน 140 มล. 1x3x10,ซี-วิท เลมอน 140 มล. 1x3x10,['C-Vitt Lemon 140 ml'],['C-Vitt Orange 140 ml'],C-VITT,C-VITT
1,Bangkok,10000600,10000602,10000603,10000399,7.052634,0.450481,ซี-วิท ออเร้นจ์ 140 มล. 1x3x10,ซี-วิท ออเร้นจ์ 140 มล. 1x3x10,['C-Vitt Orange 140 ml'],['Lipovitan-D 100 ml'],C-VITT,LIPO
2,Bangkok,10000640,10000627,10000602,10000628,6.984442,0.219047,ซี-วิท เลมอน 140 มล. 1x3x10,ซี-วิท เลมอน 140 มล. 1x3x10,['C-Vitt Lemon 140 ml'],['C-Vitt Orange 140 ml'],C-VITT,C-VITT
3,Bangkok,10000640,10000628,10000602,10000627,6.954867,0.186408,ซี-วิท เลมอน 140 มล. 1x3x10,ซี-วิท เลมอน 140 มล. 1x3x10,['C-Vitt Lemon 140 ml'],['C-Vitt Lemon 140 ml'],C-VITT,C-VITT
4,Bangkok,10000640,10000512,10000359,0,6.587455,0.421139,โสมอินซัม พลัสเห็ดหลินจือ 100มล 1x5x10,NaN,['Som in sum Lingzhi 100 ml'],NaN,SOM IN SUM,LIPO
